# Setup

In [ ]:
import os
os.cpu_count()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/

In [ ]:
# !rm -r netFound

In [ ]:
# !git clone https://github.com/SNL-UCSB/netFound.git

In [ ]:
cd netFound

In [ ]:
!apt-get update && apt-get upgrade -y
!apt-get install -y --no-install-recommends gcc python3-dev wget graphviz parallel nano make cmake g++ libpcap-dev

In [ ]:
!pip install --no-cache-dir -r requirements.txt

In [ ]:
!rm -rf '/usr/local/pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64'

In [ ]:
!wget https://github.com/seladb/PcapPlusPlus/releases/download/v24.09/pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64.tar.gz
!tar -xvf pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64.tar.gz
!rm pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64.tar.gz
!mv pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64 /usr/local/
!ln -s /usr/local/pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64 /usr/local/pcapplusplus

In [ ]:
import os
os.environ["PATH"] += os.pathsep + "/usr/local/pcapplusplus/bin"

In [ ]:
!find . -type f -name "*.sh"

In [ ]:
!chmod +x ./scripts/finetune-test.sh
!chmod +x ./scripts/pretrain-test.sh
!chmod +x ./src/pre_process/1_filter.sh
!chmod +x ./src/pre_process/2_pcap_splitting.sh
!chmod +x ./src/pre_process/3_extract_fields.sh

In [ ]:
PREPROCESS_SRC_DIR = "src/pre_process/packets_processing_src"
PREPROCESS_BUILD_DIR = "build/preprocess"
DATA_DIR = "data/test"

In [ ]:
!rm -rf {PREPROCESS_BUILD_DIR}
!rm -f src/pre_process/1_filter
!rm -f src/pre_process/3_field_extraction
!rm -rf {DATA_DIR}/pretraining/split {DATA_DIR}/pretraining/filtered {DATA_DIR}/pretraining/extracted {DATA_DIR}/pretraining/final
!rm -rf {DATA_DIR}/finetuning/split {DATA_DIR}/finetuning/filtered {DATA_DIR}/finetuning/extracted {DATA_DIR}/finetuning/final

In [ ]:
!mkdir -p {PREPROCESS_BUILD_DIR}

In [ ]:
!cmake -S {PREPROCESS_SRC_DIR} -B {PREPROCESS_BUILD_DIR}

In [ ]:
!make -C {PREPROCESS_BUILD_DIR}

In [ ]:
!cp {PREPROCESS_BUILD_DIR}/1_filter src/pre_process/1_filter

In [ ]:
!cp {PREPROCESS_BUILD_DIR}/3_field_extraction src/pre_process/3_field_extraction

In [ ]:
!chmod +x src/pre_process/1_filter
!chmod +x src/pre_process/3_field_extraction

# For Pretraining

In [ ]:
!mkdir {DATA_DIR}/pretraining/raw/0

In [ ]:
!mv {DATA_DIR}/pretraining/raw/*/*.pcap {DATA_DIR}/pretraining/raw/complete

In [ ]:
!ls {DATA_DIR}/pretraining/raw/

In [ ]:
!python3 scripts/preprocess_data.py --input_folder {DATA_DIR}/pretraining --action pretrain --tokenizer_config configs/TestPretrainingConfig.json --combined

# For Finetuning

In [ ]:
import shutil
import os

In [ ]:
pcap_names = os.listdir(f"{DATA_DIR}/finetuning/raw/0")
for i in range(len(pcap_names)):
    os.mkdir(f"{DATA_DIR}/finetuning/raw/{i}")
    shutil.move(f"{DATA_DIR}/pretraining/raw/0/{pcap_names[i]}", f"{DATA_DIR}/finetuning/raw/{i}/{pcap_names[i]}")

In [ ]:
!ls {DATA_DIR}/finetuning/raw/*

In [ ]:
!python3 ./scripts/preprocess_data.py --input_folder {DATA_DIR}/finetuning --action finetune --tokenizer_config configs/TestFinetuningConfig.json --combined

In [ ]:
!ls {DATA_DIR}/finetuning/final/*